In [ ]:
# 1. Clone the repository
!git clone https://github.com/AhsanBaidar/MuLA_GAN.git

# 2. Navigate into the folder
%cd MuLA_GAN

# 3. Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'MuLA_GAN'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 125 (delta 30), reused 78 (delta 15), pack-reused 28 (from 1)
Receiving objects: 100% (125/125), 144.94 MiB | 34.59 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/MuLA_GAN
Mounted at /content/drive


In [ ]:
# ==========================================================
# 1. USER CONFIGURATION: SET DATASET PATHS
# ==========================================================
# The base folder of your dataset (e.g., UIEB, LSUI, etc.)
BASE_PATH = '/content/drive/MyDrive/LSUI_Split'

# Relative subfolders for Train/Test
TRAIN_A = 'train/trainA'
TRAIN_B = 'train/trainB'
TEST_A = 'test/testA'
TEST_B = 'test/testB'

# Name for your experiment folder
DATASET_NAME = "LSUI-Split"

In [ ]:
import os

# 1. Create directory structure
os.makedirs("configs", exist_ok=True)
os.makedirs("utils", exist_ok=True)

# 2. Generate the YAML for the training script
yaml_content = f"""
dataset_path: '{BASE_PATH}'
TRAIN_INPUT: '{TRAIN_A}'
TRAIN_GT: '{TRAIN_B}'
TEST_INPUT: '{TEST_A}'
TEST_GT: '{TEST_B}'
dataset_name: "{DATASET_NAME}"

im_width: 256
im_height: 256
chans: 3

MODEL_NAME: 'MuLA_GAN'
GENERATOR: 'mula_gan_g'
DISCRIMINATOR: 'mula_gan_d'

BATCH_SIZE: 8
NUM_EPOCHS: 200
LR_G: 0.0002
LR_D: 0.0002
B1: 0.5
B2: 0.999
WEIGHT_DECAY: 0.0001
LAMBDA_L1: 100
LAMBDA_ADV: 1
LAMBDA_PERCEPTUAL: 10

ckpt_interval: 20
SAMPLE_INTERVAL: 1000
LOG_INTERVAL: 50
val_interval: 1000

CHECKPOINT_DIR: 'checkpoints'
SAMPLE_DIR: 'samples'
RESULTS_PATH: 'results'
"""

with open("configs/train_MuLA-GAN.yaml", "w") as f:
    f.write(yaml_content)

# 3. Create the Dataloader file
with open("utils/data_utils.py", "w") as f:
    f.write("""import os
import glob
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import yaml

class Dataloader(Dataset):
    def __init__(self, root, dataset_name, transforms_=None, config_path="configs/train_MuLA-GAN.yaml"):
        self.transform = transforms.Compose(transforms_)
        with open(config_path) as f:
            cfg = yaml.load(f, Loader=yaml.FullLoader)
        input_path = os.path.join(cfg.get("dataset_path", root), cfg.get("TRAIN_INPUT"))
        gt_path = os.path.join(cfg.get("dataset_path", root), cfg.get("TRAIN_GT"))
        self.filesA = sorted(glob.glob(input_path + "/*.*"))
        self.filesB = sorted(glob.glob(gt_path + "/*.*"))
        self.len = min(len(self.filesA), len(self.filesB))

    def __getitem__(self, index):
        idx = index % self.len
        img_A = Image.open(self.filesA[idx]).convert('RGB')
        img_B = Image.open(self.filesB[idx]).convert('RGB')
        return {"A": self.transform(img_A), "B": self.transform(img_B)}

    def __len__(self):
        return self.len
""")

print(f" Setup Complete. Files created for dataset: {DATASET_NAME}")

In [ ]:
# This will use the config created in Cell 2
!python train_MuLA_GAN.py

In [ ]:
# Define which epoch weight to test (e.g., the last one)
EPOCH_TO_TEST = 300

weights_path = f"checkpoints/{DATASET_NAME}/generator_{EPOCH_TO_TEST}.pth"
test_data_path = f"{BASE_PATH}/{TEST_A}"
output_dir = f"./output_{DATASET_NAME.lower()}/"

!python test.py \
  --weights_path "{weights_path}" \
  --data_dir "{test_data_path}" \
  --sample_dir "{output_dir}"

In [ ]:
# Create the Evaluation directory if it doesn't exist
os.makedirs("Evaluation", exist_ok=True)

# Generate the script with the variables from Cell 1
evaluation_script = f"""
import numpy as np
from PIL import Image
from glob import glob
from os.path import join, exists
from ntpath import basename
import os

try:
    from Evaluation.imqual_utils import getSSIM, getPSNR
except ImportError:
    try:
         from imqual_utils import getSSIM, getPSNR
    except ImportError as e:
        print(f"Error importing imqual_utils: {{e}}")
        exit()

def SSIMs_PSNRs(gtr_dir, gen_dir, im_res=(256, 256)):
    gtr_paths = sorted(glob(join(gtr_dir, "*.*")))
    ssims, psnrs = [], []
    processed_count = 0

    if not gtr_paths:
        print(f"Error: No images found in {{gtr_dir}}")
        return np.array([]), np.array([])

    for gtr_path in gtr_paths:
        gtr_f_base = basename(gtr_path).split('.')[0]
        # Check common formats
        gen_path = None
        for ext in [".png", ".jpg", ".jpeg"]:
            check_path = join(gen_dir, gtr_f_base + ext)
            if exists(check_path):
                gen_path = check_path
                break

        if gen_path:
            processed_count += 1
            try:
                r_im = Image.open(gtr_path).resize(im_res).convert('RGB')
                g_im = Image.open(gen_path).resize(im_res).convert('RGB')

                ssim_val = getSSIM(np.array(r_im), np.array(g_im))
                if np.isfinite(ssim_val): ssims.append(ssim_val)

                psnr_val = getPSNR(np.array(r_im.convert("L")), np.array(g_im.convert("L")))
                if np.isfinite(psnr_val): psnrs.append(psnr_val)
            except Exception as e:
                 print(f"Error processing {{basename(gtr_path)}}: {{e}}")

    return np.array(ssims), np.array(psnrs)

# Using variables defined in Cell 1
gtr_dir = "{BASE_PATH}/{TEST_B}"
gen_dir = "./output_{DATASET_NAME.lower()}/"

SSIM_measures, PSNR_measures = SSIMs_PSNRs(gtr_dir, gen_dir)

if len(SSIM_measures) > 0:
    print(f"\\n--- {{os.path.basename(gen_dir)}} Results ---")
    print(f"SSIM ({{len(SSIM_measures)}} samples) -> Mean: {{np.mean(SSIM_measures):.4f}}")
    print(f"PSNR ({{len(PSNR_measures)}} samples) -> Mean: {{np.mean(PSNR_measures):.2f}}")
"""

with open("Evaluation/measure_ssim_psnr.py", "w") as f:
    f.write(evaluation_script)

# Execute the script
!python Evaluation/measure_ssim_psnr.py

In [ ]:
# Generate the UIQM script using the global output directory
uiqm_script = f"""
import numpy as np
from PIL import Image
from glob import glob
from os.path import join
from ntpath import basename
import os

try:
    from Evaluation.uqim_utils import getUIQM
except ImportError:
    try:
         from uqim_utils import getUIQM
    except ImportError as e:
        print(f"Error importing uqim_utils: {{e}}")
        exit()

def measure_UIQMs(dir_name, im_res=(256, 256)):
    paths = sorted(glob(join(dir_name, "*.*")))
    uqims = []
    if not paths:
        print(f"Error: No images found in directory: {{dir_name}}")
        return np.array([])
    for img_path in paths:
        try:
            im = Image.open(img_path).resize(im_res).convert('RGB')
            uiqm = getUIQM(np.array(im))
            if np.isfinite(uiqm):
                 uqims.append(uiqm)
        except Exception as e:
             print(f"Error processing {{basename(img_path)}}: {{e}}")
    return np.array(uqims)

# Auto-populated from Cell 1 variables
gen_dir = "./output_{DATASET_NAME.lower()}/"

print(f"Evaluating UIQM for: {{gen_dir}}")
gen_uqims = measure_UIQMs(gen_dir)

if len(gen_uqims) > 0:
    print("\\n--- UIQM Results ---")
    print(f"Mean UIQM ({{len(gen_uqims)}} samples): {{np.mean(gen_uqims):.4f}}")
    print(f"Std Dev: {{np.std(gen_uqims):.4f}}")
"""

with open("Evaluation/measure_uiqm.py", "w") as f:
    f.write(uiqm_script)

# Run the evaluation
!python Evaluation/measure_uiqm.py